In [1]:
# ~~~~~~~~~~~~~~ Libraries
import sys, os
import mne # Python package for processing and analyzing electrophysiological data
import numpy as np
from glob import glob # look for all the pathnames matching a specified pattern according to the rules
import matplotlib.pyplot as plt
from mne.preprocessing import ICA # ICA (Independent Component Analysis) algorithm, which is for artifact removal
from autoreject import AutoReject # Python package for automatically rejecting bad epochs in EEG/MEG data
import json
import owncloud
import pandas as pd
import braindecode
import torch
import re
import pickle
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict, cross_validate, StratifiedKFold

# Deep learning
from braindecode.models import EEGNetv4
from braindecode.preprocessing import exponential_moving_standardize
from braindecode.util import set_random_seeds
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier
# ~~~~~~~~~~~~~~ Libraries ~~~~~~~~~~~~~~

In [7]:
# ~~~~~~~~~~~~~~ Parameters
group = 'adult'
modality = 'visual' # 'visual' or 'audio'
subject = 'sub-03'
condition = 'singledot' # 'singledot', 'totaldot', 'circum'

# EEGNET parameters
# Load the JSON file
code_directory = os.path.dirname("/u/kazma/MINT/code/1st-level")
sys.path.append(code_directory) 
os.chdir(os.path.dirname("/u/kazma/MINT/code/1st-level"))
print(os.getcwd())
with open("config.json") as f: # import variables from config.json
    config = json.load(f) 
globals().update(config)


# Print out each parameter
print(f"{modality} data of {subject} is processed")
print(f"condition: {condition}")
print("EEGNET parameters:")
print(f"====== CV: {EEGNET_CV}")
print(f"====== Max epochs: {EEGNET_MAX_EPOCHS}")
print(f"====== Batch size: {EEGNET_BATCH_SIZE}")

/viper/u/kazma/MINT/code
visual data of sub-03 is processed
condition: singledot
EEGNET parameters:
====== CV: 10
====== Max epochs: 200
====== Batch size: 16


In [8]:
# ~~~~~~~~~~~~~~ Set the working directory
path = f"{COMPUTE_DIR}/data/{group}/interim/{modality}"
sub_folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
sub_folders_sorted = sorted(sub_folders, key=lambda x: int(re.search(r'\d+', x).group())) # Sort the folders based on the numeric part after "sub-"
# ~~~~~~~~~~~~~~ Set the working directory ~~~~~~~~~~~~~~

# ~~~~~~~~~~~~~~ Concatanating 3 sessions
each_sub_path = f"{COMPUTE_DIR}/data/{group}/interim/{modality}/{subject}"
each_sub_folders = [f for f in os.listdir(each_sub_path) if os.path.isdir(os.path.join(each_sub_path, f))]
each_sub_folders_sorted = sorted(each_sub_folders, key=lambda x: int(re.search(r'\d+', x).group()))

eegdata_dict = {}
for x,ses_loop in enumerate(each_sub_folders_sorted):
    sub_filename = os.path.join(path, subject, ses_loop, 'epochs-epo.fif') 
    epochs = mne.read_epochs(sub_filename, preload=True)
    eegdata_dict[ses_loop] = epochs

# concatanate all 3 sessions into 1 epoch
epochs = mne.concatenate_epochs([eegdata_dict['ses-01'], eegdata_dict['ses-02'], eegdata_dict['ses-03']])
# ~~~~~~~~~~~~~~ Concatanating 3 sessions ~~~~~~~~~~~~~~



Reading /ptmp/kazma/DATA-MINT/data/adult/interim/visual/sub-03/ses-01/epochs-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1260 matching events found
No baseline correction applied
0 projection items activated
Reading /ptmp/kazma/DATA-MINT/data/adult/interim/visual/sub-03/ses-02/epochs-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1260 matching events found
No baseline correction applied
0 projection items activated
Reading /ptmp/kazma/DATA-MINT/data/adult/interim/visual/sub-03/ses-03/epochs-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1260 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_1109483/305156617.py:19: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([eegdata_dict['ses-01'], eegdata_dict['ses-02'], eegdata_dict['ses-03']])


Not setting metadata
3780 matching events found
Applying baseline correction (mode: mean)


### Get controled stimuli parameter vector

In [9]:
# ~~~~~~~~~~~~~~ Concatanating 3 sessions csv file
each_sub_path = f"{COMPUTE_DIR}/data/{group}/raw/{modality}/{subject}"
each_sub_folders = [f for f in os.listdir(each_sub_path) if os.path.isdir(os.path.join(each_sub_path, f))]
each_sub_folders_sorted = sorted(each_sub_folders, key=lambda x: int(re.search(r'\d+', x).group()))

csv_dict = {}
for x,ses_loop in enumerate(each_sub_folders_sorted):
    csv_file = f"{subject}_{modality}_{ses_loop}.csv"
    sub_filename = os.path.join(each_sub_path, ses_loop, csv_file) 
    csv_df = pd.read_csv(sub_filename)
    csv_dict[ses_loop] = csv_df

# concatanate all 3 sessions into 1 epoch
csv_dfs = pd.concat([csv_dict['ses-01'], csv_dict['ses-02'], csv_dict['ses-03']], axis=0)
# ~~~~~~~~~~~~~~ Concatanating 3 sessions csv file~~~~~~~~~~~~~~



# ~~~~~~~~~~~~~~ Obtain target conditions' trials
csv_dfs['Extracted'] = csv_dfs['condition'].str.split('_').str[-1]
csv_idx = csv_dfs[['condition', 'Extracted']]
condition_lists = csv_idx['Extracted'].unique()

# target_condition = condition_lists[0]
target_condition = condition

idx_to_keep = csv_dfs.index[csv_dfs['Extracted'] == target_condition].tolist()
selected_epochs = epochs[idx_to_keep]
print(f"Selected trials: {len(selected_epochs)}")
print(f"Target condition: {target_condition}")
# ~~~~~~~~~~~~~~ Obtain target conditions' trials ~~~~~~~~~~~~~~

Selected trials: 1260
Target condition: singledot


In [10]:
# Crop epochs to the desired time range
cropped_epochs = selected_epochs.copy().crop(tmin=EEGNET_MIN_TIME, tmax=EEGNET_MAX_TIME)

# Get the info about the cropped data
conditions = list(cropped_epochs.event_id.keys()) # list of conditions
n_conditions = len(conditions) # number of conditions
n_trials = len(cropped_epochs) # number of trials
n_samples = cropped_epochs.get_data().shape[2]
n_channels = cropped_epochs.get_data().shape[1]
min_time = cropped_epochs.times[0]*1000   # First time point in milli seconds
max_time = cropped_epochs.times[-1]*1000    # Last time point in milli seconds


print("=====================================================")
print("=====================================================")
print("=====================================================")

print(f" Condition lists: {conditions}")
print(f" Total trials: {n_trials}")
print(f" Time points: {n_samples}")
print(f" Time window (ms): {min_time} - {max_time}")


# check GPU availability
cuda = torch.cuda.is_available()  # Check if a GPU is available
device = "cuda" if cuda else "cpu"  # Use "cuda" if available, otherwise fallback to "cpu"

if cuda:
    torch.backends.cudnn.benchmark = True  # Enable cuDNN auto-tuning for performance


# Set random seeds
# PURPOSE: reproducibility across runs, random initializations (e.g., model weights) yield consistant results
seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)


# dictionaries for decoding accuracy
pairwise_decoding_accuracies = {}
# dictionaries for decoding accuracy of standard deviation
pairwise_decoding_accuracies_std = {}
# dictionaries for estimators
pairwise_decoding_estimators = {}


 Condition lists: ['numerosity 1', 'numerosity 2', 'numerosity 3', 'numerosity 4', 'numerosity 5', 'numerosity 6']
 Total trials: 1260
 Time points: 176
 Time window (ms): 0.0 - 350.0


In [10]:
for i in range(n_conditions):
    for j in range(i + 1, n_conditions):

            cond1 = conditions[i]
            cond2 = conditions[j]

            print(f"{cond1} vs. {cond2}")
            
            filtered_epochs = cropped_epochs[cond1,cond2]

            # get EEG data
            X = filtered_epochs.get_data(picks=mne.pick_types(filtered_epochs.info, eeg=True, eog=False, exclude='bads'))

            # get labels (=numerosity)
            unique_labels = np.unique(filtered_epochs.events[:,-1])
            label_0, label_1 = unique_labels[0], unique_labels[1]  # Assign the first label to 0 and the second to 1

            y = np.where(filtered_epochs.events[:, -1] == label_0, 0, 1)

            skfold = StratifiedKFold(n_splits=EEGNET_CV, shuffle=True, random_state=23)

            # exp. moving std. for each trial
            for s in range(X.shape[0]):
                X[s,:,:] = exponential_moving_standardize(X[s,:,:], factor_new=0.001, init_block_size=None, eps=1e-4)

            # create the model
            net = EEGClassifier(
                "EEGNetv4", 
                            module__n_chans=n_channels, # Number of EEG channels
                            module__n_outputs=2,               # Number of outputs of the model. This is the number of classes in the case of classification.
                            module__n_times=n_samples,         # Number of time samples of the input window.
                            module__final_conv_length='auto',  # Length of the final convolution layer. If "auto", it is set based on the n_times.
                            module__pool_mode='mean',          # Pooling method to use in pooling layers
                            module__F1=8,                      # Number of temporal filters in the first convolutional layer.
                            module__D=2,                       # Depth multiplier for the depthwise convolution.
                            module__F2=16,                     # Number of pointwise filters in the separable convolution. Usually set to ``F1 * D``.
                            module__kernel_length=64,         # Length of the temporal convolution kernel. Usally sampling rate / 2 = 500/2 = 250
                            module__third_kernel_size=(8, 4), 
                            module__drop_prob=0.25,            # Dropout probability after the second conv block and before the last layer. 0.5 for within-subject classification, 0.25 in cross-subject clasification
                            module__chs_info=None,             # (list of dict) – Information about each individual EEG channel. This should be filled with info["chs"]. Refer to mne.Info for more details.
                            module__input_window_seconds=None, # Length of the input window in seconds.
                            module__sfreq=SFREQ,
                            max_epochs=EEGNET_MAX_EPOCHS,
                            batch_size=EEGNET_BATCH_SIZE,
                            train_split=None,
            )


            cvs = cross_validate(net, 
                                X, 
                                y, 
                                scoring="accuracy", # for balanced classes, this corresponds to accuracy,
                                # chance level might be 0 (adjusted = False), or 0.X (adjusted = True)
                                cv=skfold, 
                                n_jobs=-1, # only 1 to avoid overload in parallel jobs, in non-par jobs it could be -1
                                return_estimator=True, # if you need the model to estimate on another test set
                                return_train_score=False,
                                )
            
            pairwise_decoding_accuracies[(cond1,cond2)] = np.mean(cvs['test_score'])
            pairwise_decoding_accuracies_std[(cond1,cond2)] = np.std(cvs['test_score'])
            pairwise_decoding_estimators[(cond1,cond2)] = cvs['estimator']


# ~~~~~~~~~~~~~~~~ Save the decoding accuracy
save_folder = f"{COMPUTE_DIR}/data/{group}/processed/{modality}/{subject}"

filename = f"EEGNet_accuracy_pairwise_{target_condition}.pkl"
save_path = os.path.join(save_folder, filename) #  a pickle file
with open(save_path, "wb") as f:
    pickle.dump(pairwise_decoding_accuracies, f)
print(f"{subject}: saved in {filename}")
# ~~~~~~~~~~~~~~~~ Save the decoding accuracy ~~~~~~~~~~~~~~~~

# ~~~~~~~~~~~~~~~~ Save the decoding accuracy standard deviation
filename = f"EEGNet_std_pairwise_{target_condition}.pkl"
save_path = os.path.join(save_folder, filename) #  a pickle file
with open(save_path, "wb") as f:
    pickle.dump(pairwise_decoding_accuracies_std, f)
print(f"{subject}: saved in {filename}")
# ~~~~~~~~~~~~~~~~ Save the decoding accuracy standard deviation ~~~~~~~~~~~~~~~~


# ~~~~~~~~~~~~~~~~ Save the EEGNet estimation
filename = f"EEGNet_estimator_pairwise_{target_condition}.pkl"
save_path = os.path.join(save_folder, filename) #  a pickle file
with open(save_path, "wb") as f:
    pickle.dump(pairwise_decoding_estimators, f)
print(f"{subject}: saved in {filename}")
# ~~~~~~~~~~~~~~~~ Save the EEGNet estimation ~~~~~~~~~~~~~~~~

numerosity 1 vs. numerosity 2


/viper/ptmp/kazma/miniforge3/envs/MINT/lib/python3.9/site-packages/skorch/net.py:2261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cuda_attrs = torch.load(f, **load_kwargs

  epoch    train_loss     dur
-------  ------------  ------
      1        0.7848  2.3062
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7402  2.5016
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7990  2.2046
  epoch    train_loss     dur
-------  ------------  ------
      1        0.8220  2.1012
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7426  2.3130
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7543  2.4176
  epoch    train_loss     dur
-------  ------------  ------
      1        0.8341  2.4907
  epoch    train_loss     dur
-------  ------------  ------
      1        0.8070  2.5184
      2        0.6805  2.0991
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7930  3.8926
      2        0.7179  2.2078
      2        0.7195  2.0137
      2        0.7113  2.3065
      2        0.6974  2.2166
  epoch    train_loss     dur
-------  -

/viper/ptmp/kazma/miniforge3/envs/MINT/lib/python3.9/site-packages/skorch/net.py:2261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cuda_attrs = torch.load(f, **load_kwargs

    155        0.1210  2.5909
    165        0.0929  2.4928
    159        0.1260  2.5954
    170        0.1209  2.4027
    162        0.1287  2.4054
    176        0.0938  2.1013
    169        0.0938  2.3884
    175        0.1047  2.2818
    171        0.0961  2.2975
    156        0.0811  2.4042
    166        0.1171  2.1095
    171        0.1166  2.1073
    160        0.1369  2.3925
    177        0.0934  2.0885
    163        0.1213  2.2837
    176        0.0942  2.0148
    170        0.1041  2.2028
    172        0.0732  2.1085
    157        0.1034  2.2947
    167        0.1076  2.1937
    172        0.1341  2.1095
    161        0.1518  2.2060
    178        0.1125  2.0050
    177        0.0696  2.0940
    164        0.1032  2.2059
    171        0.1180  2.1146
    173        0.0720  2.2919
    158        0.1086  2.3030
    168        0.1281  2.0075
    173        0.1410  2.0938
    162        0.1183  2.3842
    179        0.1138  2.0940
    178        0.1078  2.1016
    172   

...

  epoch    train_loss     dur
-------  ------------  ------
      1        0.7488  3.7018
      2        0.6840  3.6971
      3        0.6293  3.6021
      4        0.6101  4.0007
      5        0.5951  4.0964
      6        0.5720  3.9015
      7        0.5500  4.0023
      8        0.5418  4.0984
      9        0.5602  3.8997
     10        0.5318  4.1973
     11        0.5400  3.8982
     12        0.5136  4.0986
     13        0.5360  3.8971
     14        0.5020  3.6085
     15        0.4936  3.7992
     16        0.4803  4.2956


/viper/ptmp/kazma/miniforge3/envs/MINT/lib/python3.9/site-packages/skorch/net.py:2261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cuda_attrs = torch.load(f, **load_kwargs

     17        0.4814  4.0970
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7922  4.5048
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7549  4.7060
     18        0.4698  4.8029
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7889  4.9014
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7816  4.9059
  epoch    train_loss     dur
-------  ------------  ------
      1        0.8573  4.9074
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7539  4.9046
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7985  5.2011
  epoch    train_loss     dur
-------  ------------  ------
      1        0.8211  5.8011
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7318  5.7978
     19        0.4385  4.3025
      2        0.6949  4.6020
      2        0.6984  4.3982
      2        0.7045  4.2967
      2   

/viper/ptmp/kazma/miniforge3/envs/MINT/lib/python3.9/site-packages/skorch/net.py:2261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cuda_attrs = torch.load(f, **load_kwargs

    178        0.1110  3.9030
    181        0.0952  3.8976
    150        0.1069  4.7952
    186        0.0868  3.9941
    183        0.0674  3.8973
    177        0.0990  4.0998
    153        0.1090  4.1010
    174        0.0847  3.7080
    173        0.1156  3.6049
    179        0.1240  3.5997
    182        0.0943  3.7054
    187        0.1140  3.3999
    184        0.0677  3.6042
    178        0.1208  3.8028
    151        0.1145  4.4998
    175        0.0977  3.6950
    154        0.1057  4.3969
    174        0.0953  3.6928
    180        0.0832  3.5981
    183        0.1247  3.7939
    188        0.0957  3.6043
    185        0.0713  3.6984
    179        0.1220  4.1048
    176        0.0997  3.9030
    152        0.1056  5.0971
    175        0.1184  3.9066
    181        0.1024  3.8047
    184        0.0917  3.7964
    155        0.1158  4.9998
    189        0.0840  3.7011
    186        0.1050  3.8963
    180        0.1022  3.9977
    177        0.0887  3.6002
    176   